In [3]:
from routines import reset_all_trades, create_new_investor, create_new_stock
from constants import name_id, id_name
import pandas as pd
from bank import buy_stock, sell_stock, pay_all_dividends
from visual import print_all
from prestige_hype import compute_prestige_and_hype
from routines import refresh_player_data_raw
from visual import add_current_name_col
pd.set_option('display.width', 1800)

# TODO:
# new_season function propre avec broadcast message


#### New season start

In [6]:
# do not use reset_all_trades() until new season.

# # create traders
# create_new_investor('besta', 99999999)
# create_new_investor('carbone', 1000)
# create_new_investor('mion1', 1000)


# create all stocks, once
# await refresh_player_data_raw()
# df = compute_prestige_and_hype()
# for i,x in enumerate(df.index):
#     pp,p,h = df.loc[x,:]
#     create_new_stock(x, pp, h, p, 1000, 0)

In [ ]:
# Ideally every 10 minutes, update all stocks
from datetime import datetime
import time
from routines import update_stock, refresh_player_data_raw
from utils import get_stock_by_name
from prestige_hype import compute_prestige_and_hype


# for _ in range(99999):
#     try:
#         refresh_player_data_raw()
#         df = compute_prestige_and_hype()
#         for x in df.index:
#             pp,p,h = df.loc[x,:]
#             stock = get_stock_by_name(x)
#             stock.raw_skill = pp 
#             stock.trendiness = h
#             stock.prestige = p
#             update_stock(stock)
#     except Exception as e:
#         print(datetime.now(), e)
#         time.sleep(60)
#         continue
#     time.sleep(300)

In [ ]:
# from osuapi import api
# from datetime import timedelta

# u = api.user(1019489)


